In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import seaborn as sns
import numpy
from torch.utils.data import DataLoader, Dataset, random_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from collections import defaultdict, Counter

In [2]:
train_data = pd.read_csv('~/Datasets/train.csv', index_col = 'Id')
test_data = pd.read_csv('~/Datasets/test.csv', index_col = 'Id')

In [3]:
train_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
test_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [5]:
columns = list(train_data.columns.values)
name_to_index = defaultdict(int)
for i,name in enumerate(columns):
    name_to_index[name] = i

In [6]:
train_data2 = train_data.applymap(lambda x: isinstance(x, str) or pd.isnull(x))

In [7]:
nulls = []
for i in range(len(train_data2.iloc[0])):
    if all(train_data2.iloc[:, i]):
        nulls.append(i)

In [8]:
arr = ["Street", "Alley", "PoolQC", "Fence", "MiscFeature", "MSZoning", "LandContour", "Utilities", "FireplaceQu", "RoofMatl", "LandSlope", "BsmtFinType2", "PavedDrive", "Condition1","Exterior2nd","Neighborhood","Condition2","ExterCond", "MasVnrType","BsmtFinType1", "BsmtCond", "BldgType", "LotConfig", "Exterior1st", "SaleType"]
nums = []
for a in arr:
    nums.append(name_to_index[a])
    del train_data[a]
    del test_data[a]
for num in nums:
    nulls.remove(num)
    


In [9]:
train_data = train_data.fillna(train_data.mode().iloc[0])
test_data = test_data.fillna(test_data.mode().iloc[0])

In [10]:
changed_columns = set()
for n in nulls:
    
    # for train_data 
    name = columns[n]
    data = pd.DataFrame(train_data[name], columns = [name])
    # creating instance of labelencoder
    labelencoder = LabelEncoder()
    # Assigning numerical values and storing in another column
    data[name+'Cat'] = labelencoder.fit_transform(data[name]) 
    del train_data[name]
    train_data[name+'Cat'] = data[name+'Cat']
    
    # for test_data
    data2 = pd.DataFrame(test_data[name], columns = [name])
    # creating instance of labelencoder
    labelencoder = LabelEncoder()
    # Assigning numerical values and storing in another column
    data2[name+'Cat'] = labelencoder.fit_transform(data2[name]) 
    del test_data[name]
    test_data[name+'Cat'] = data2[name+'Cat']
    
    changed_columns.add(name+'Cat')
    

In [11]:
for column in changed_columns:
    # for train_data
    enc = OneHotEncoder(handle_unknown = 'ignore')
    enc_df = pd.DataFrame(enc.fit_transform(train_data[[column]]).toarray())
    l = len(list(enc_df.columns.values))
    mapping = defaultdict(int)
    name = column[:-3]
    for i in range(l):
        mapping[i] = name+str(i)
    enc_df = enc_df.rename(columns = mapping)
    enc_df.index += 1
    train_data = train_data.join(enc_df)

In [12]:
for column in changed_columns:
    # for test_data
    enc = OneHotEncoder(handle_unknown = 'ignore')
    enc_df = pd.DataFrame(enc.fit_transform(test_data[[column]]).toarray())
    l = len(list(enc_df.columns.values))
    mapping = defaultdict(int)
    name = column[:-3]
    for i in range(l):
        mapping[i] = name+str(i)
    enc_df = enc_df.rename(columns = mapping)
    enc_df.index += 1461
    test_data = test_data.join(enc_df)

In [13]:
train_data = train_data.to_numpy()
test_data = test_data.to_numpy()

In [28]:
test_data.shape

(1459, 139)

In [14]:
train_mean = numpy.mean(train_data, axis = 0)
train_std = numpy.std(train_data, axis = 0)

In [15]:
class Dataset(Dataset):
    def __init__(self,train_data = train_data):
        self.train_data = train_data
    def __len__(self):
        return len(self.train_data)
    def __getitem__(self, i):
        data = self.train_data[i]
        x =  ((torch.tensor(data[:-1].astype(float),dtype=torch.float) - train_mean[:-1])/train_std[:-1]).float()
        x = torch.tensor(data[:-1], dtype = torch.float)
        y = torch.tensor(data[-1], dtype = torch.float)
        return x, y

In [16]:
train_set, test_set = random_split(Dataset(train_data),[1100,360])

In [17]:
train_set

In [18]:
train_loader = DataLoader(train_set, shuffle=True, batch_size = 5)
test_loader = DataLoader(test_set, batch_size = 2)

In [19]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.first_linear = nn.Linear(144, 148)
        self.second_linear = nn.Linear(148, 110)
        self.third_linear = nn.Linear(110, 80)
        self.forth_linear = nn.Linear(80, 50)
        self.fifth_linear = nn.Linear(50, 1)
        self.bn1 = nn.BatchNorm1d(148)
        self.bn2 = nn.BatchNorm1d(110)
        self.bn3 = nn.BatchNorm1d(80)
        self.bn4 = nn.BatchNorm1d(50)
        self.dropout = nn.Dropout(p = 0.4)
        
    def forward(self, batch):
        pred = self.first_linear(batch)
        pred = self.bn1(pred)
        pred = torch.relu(pred)
        pred = self.second_linear(pred)
        pred = self.bn2(pred)
        pred = torch.relu(pred)
        pred = self.dropout(pred)
        pred = self.third_linear(pred)
        pred = self.bn3(pred)
        pred = torch.relu(pred)
        pred = self.forth_linear(pred)
        pred = self.bn4(pred)
        pred = torch.relu(pred)
        pred = self.fifth_linear(pred)
        return torch.squeeze(pred)
        

In [20]:
model = Model()

In [21]:
optimizer = optim.Adam(model.parameters(),lr = 0.002)
criterion = nn.MSELoss()

In [22]:
x = []
y = []
for epoch in range(50):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
            
# for epoch in range(10):
#     optimizer.zero_grad()
#     outputs = model(x)
# #     print(outputs)
#     loss = criterion(outputs, y)
#     loss.backward()
#     optimizer.step()
#     print(loss)

[1,   100] loss: 0.046
[1,   200] loss: 0.014
[2,   100] loss: 0.009
[2,   200] loss: 0.003
[3,   100] loss: 0.005
[3,   200] loss: 0.003
[4,   100] loss: 0.003
[4,   200] loss: 0.005
[5,   100] loss: 0.005
[5,   200] loss: 0.001
[6,   100] loss: 0.004
[6,   200] loss: 0.005
[7,   100] loss: 0.005
[7,   200] loss: 0.005
[8,   100] loss: 0.004
[8,   200] loss: 0.004
[9,   100] loss: 0.004
[9,   200] loss: 0.002
[10,   100] loss: 0.002
[10,   200] loss: 0.007
[11,   100] loss: 0.005
[11,   200] loss: 0.004
[12,   100] loss: 0.007
[12,   200] loss: 0.003
[13,   100] loss: 0.002
[13,   200] loss: 0.007
[14,   100] loss: 0.006
[14,   200] loss: 0.002
[15,   100] loss: 0.000
[15,   200] loss: 0.008
[16,   100] loss: 0.004
[16,   200] loss: 0.002
[17,   100] loss: 0.002
[17,   200] loss: 0.004
[18,   100] loss: 0.006
[18,   200] loss: 0.002
[19,   100] loss: 0.006
[19,   200] loss: 0.000
[20,   100] loss: 0.002
[20,   200] loss: 0.006
[21,   100] loss: 0.004
[21,   200] loss: 0.004
[22,   100

In [23]:
val_loader = DataLoader(test_data, batch_size = 2)